In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from functions import *

import QuantLib as ql
import pandas as pd
import numpy as np

In [45]:
discount_curve = pd.read_excel("tsData2025.xlsx", index_col = 0)
# new_row = pd.DataFrame([[1]], columns=discount_curve.columns)
# discount_curve = pd.concat([new_row, discount_curve], axis = 0, ignore_index= True)

In [46]:
notional = 100
coupon_rate = 0.03
interest_rate = 0.04

calendar = ql.TARGET()

settlement_date = ql.Date(12,11,2025)
maturity_date = ql.Date(12,5,2027)

tenor = ql.Period(1,ql.Years)
day_count = ql.ActualActual(ql.ActualActual.ISDA)
day_rolling = ql.ModifiedFollowing
termination_day_rolling = ql.ModifiedFollowing
end_of_month = False
date_generation = ql.DateGeneration.Backward



schedule = ql.Schedule( settlement_date,
                        maturity_date,
                        tenor,
                        calendar,
                        day_rolling,
                        termination_day_rolling,
                        date_generation,
                        end_of_month)   

In [47]:
price = price_coupon_bond(
    settlement_date,
    maturity_date,
    discount_curve,
    calendar,
    tenor,
    coupon_rate,
    notional,
    day_count,
    day_rolling,
    schedule
)

In [48]:
dirty_price = price["Dirty Price"]

print(f"The price of the bond is {np.round(dirty_price,2)}.")

The price of the bond is 105.48.


In [49]:
price["Summary Table"]

,Start Date,Pay Date,Year Frac,Pay Time,Discount Curve,Cash Flows,Discounted Cash Flows
1,"May 12th, 2025","May 12th, 2026",1.0,0.49589,0.998950,3.0,2.996849
2,"May 12th, 2026","May 12th, 2027",1.0,1.49589,0.995024,103.0,102.487445
